In [1]:
# !pip3 install selenium
# !pip3 install python-time
# !pip3 install beautifulsoup4
# !pip3 install wget


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
from random import shuffle
import wget


In [3]:

def GetURL(): 
    global count
    page_source = BeautifulSoup(driver.page_source)

    
    results= page_source.find_all('li' , class_='reusable-search__result-container')

    all_profiles=[]
    
    for result in results:
        spanprofile = result.find('span' , class_='entity-result__title-text')
        profile=spanprofile.find('a' , class_='app-aware-link')
        spans=profile.find_all('span')
        prof_ID=profile.get('href')
        subtitle = result.find('div' , class_='entity-result__primary-subtitle')
        image = result.find('img' , class_='presence-entity__image')
        secsubtitle = result.find('div' , class_='entity-result__secondary-subtitle')
        count+=1
        if len(spans)==0:
            pass
        else:
            name = spans[0].find_all('span')[0].text
            firstname= name.split(' ')[0]
            lastname= name.split(' ')[1]
            if subtitle is not None:
                job= subtitle.text
                job=job.replace('\n','')
            else: job='Blank'
            if secsubtitle is not None:    
                loc= secsubtitle.text
                loc=loc.replace('\n','')
            else: loc='Blank'
            if image is not None: 
                img = image.get('src')
            else: img='Blank'
            all_profiles.append([firstname, lastname, job, loc, img])       
    return all_profiles


In [4]:
# Step 1: login to LinkedIn

#Open Chrome and Login LinkedIn Login site
driver=webdriver.Chrome()
url = 'https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin'


#Get Login Credentials saved in text file in the same directory
credential = open('login_credentials.txt')
line=credential.readlines()
uname=line[0]
psd = line[1]

# Open the URL
driver.get(url)
driver.maximize_window()
sleep(2)

#Get Login Field and entering values in login text box
email=driver.find_element_by_id('username')
email.send_keys(uname)
sleep(2)

#Get Password Field and entering values in password text box
pwd=driver.find_element_by_id('password')
pwd.send_keys(psd)
sleep(2)

#Click the Signin Button
signin = driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
signin.click()
sleep(2)

search_bar= driver.find_element_by_xpath('//*[@id="global-nav-typeahead"]/input')
search_bar.send_keys('hubspot company')
sleep(3)
search_bar.send_keys(Keys.RETURN)

sleep(2)
url2='https://www.linkedin.com/company/hubspot/'
driver.get(url2)


sleep(2)
url3='https://www.linkedin.com/search/results/people/?currentCompany=%5B%2268529%22%5D&origin=COMPANY_PAGE_CANNED_SEARCH&lipi=urn%3Ali%3Apage%3Ad_flagship3_company%3BBNLZg0q1SGWsjcTVnMK5Og%3D%3D'
driver.get(url3)
sleep(2)
number_of_page=20
total_profiles=[]
count=0
for page in range(1,number_of_page):
    print(page)
    cur_pg_prof=GetURL()
    total_profiles= total_profiles +cur_pg_prof
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/5);")
    sleep(3)
    next_button = driver.find_element_by_class_name('artdeco-pagination__button--next')
    next_button.click()
    sleep(3)

shuffle(total_profiles)
df2 = pd.DataFrame(total_profiles[0:100], columns=['FirstName', 'LastName', 'Job Title', 'Location', 'Image'])
for index, row in df2.iterrows():
    img_name='Images/'+row['FirstName']+row['LastName']+'.png'
    if row['Image'] != 'Blank':
        wget.download(row['Image'], img_name)
df2.to_excel('Employees Data.xlsx', index = False)

<ipython-input-4-044142eb2552>:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  email=driver.find_element_by_id('username')
<ipython-input-4-044142eb2552>:25: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pwd=driver.find_element_by_id('password')
<ipython-input-4-044142eb2552>:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  signin = driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
<ipython-input-4-044142eb2552>:34: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar= driver.find_element_by_xpath('/html/body/div[5]/header/div/div/div/div[1]/input')


1


<ipython-input-4-044142eb2552>:57: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_button = driver.find_element_by_class_name('artdeco-pagination__button--next')


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
100% [................................................................................] 3769 / 3769

In [5]:
df2.iloc[:100]

,FirstName,LastName,Job Title,Location,Image
0,Yamini,Rangan,Chief Executive Officer at HubSpot,San Francisco Bay Area,https://media-exp1.licdn.com/dms/image/C4E03AQ...
1,Emily,Gu,Senior Frontend Developer at HubSpot,United States,https://media-exp1.licdn.com/dms/image/C4E03AQ...
2,Tiffany,Tooley,Head Of Product Marketing at HubSpot,Atlanta Metropolitan Area,https://media-exp1.licdn.com/dms/image/C5603AQ...
3,Vishal,Rajani,Technical Lead at HubSpot,"Dallas, TX",https://media-exp1.licdn.com/dms/image/C5103AQ...
4,Stefanie,Brady,Technical Recruiter at HubSpot - We're hiring!,"Sarasota, FL",https://media-exp1.licdn.com/dms/image/C4D03AQ...
...,...,...,...,...,...
95,Julie,Leist,Recruiting Coordinator at HubSpot,"Boston, MA",https://media-exp1.licdn.com/dms/image/C5603AQ...
96,Janki,S.,Talent Sourcer at HubSpot -- We're hiring!,"Miami, FL",https://media-exp1.licdn.com/dms/image/D4D35AQ...
97,Henry,Cannon,Growth Consultant at HubSpot,Kansas City Metropolitan Area,https://media-exp1.licdn.com/dms/image/C4E03AQ...
98,Nicole,Pristin,Senior Software Engineer at HubSpot,Greater Boston,https://media-exp1.licdn.com/dms/image/C4E03AQ...


In [ ]:
# total_profiles